In [ ]:
import numpy as np
import pandas as pd
import json
import urllib.request as urlreq
import os
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.sql.functions import col, count, mean, sum, avg, stddev, min, max, lit
from pyspark.ml.stat import Summarizer
from pyspark.ml.classification import LogisticRegression

In [ ]:
spark.sql("use real_world_data_ed_omop_aug_2024")

In [ ]:
def json_to_qry(url_to_json):
    json_url = urlreq.urlopen(url_to_json)
    json_file = json.loads(json_url.read())
    qry_lst = []
    def add_quote(lst):
        lst_quote = ["'"+str(x)+"'" for x in lst]
        return (lst_quote)
    for k,v in json_file.items():
        for cd,sig in v.items():
            if cd=='long': continue
            # entail the range
            if 'range' in sig:
                for x in sig['range']:
                    key_quote = [str(y) for y in list(range(int(x.split('-')[0]),int(x.split('-')[1])+1))]
                    sig['exact'].extend(key_quote)

            # generate dynamic queries
            qry = '''
                select ''' + "'" + k + "'" + ''' as CD_GRP, 
                       ''' + "'" + v['long'] + "'" + ''' as CD_GRP_LONG,
                       concept_id,concept_name,concept_code,vocabulary_id,domain_id
                from concept
                where vocabulary_id = '''+ "'" + cd.upper() + "'" +''' and
            '''
            if 'icd' in cd and 'pcs' not in cd:
                where_lev0 = '''substring_index(concept_code,'.',1) in ('''+ ','.join(add_quote(sig['lev0'])) +''')''' if sig['lev0'] else None
                where_lev1 = '''substring(concept_code,1,5) in ('''+ ','.join(add_quote(sig['lev1'])) +''')''' if sig['lev1'] else None
                where_lev2 = '''substring(concept_code,1,6) in ('''+ ','.join(add_quote(sig['lev2'])) +''')''' if sig['lev2'] else None
                where_nonempty = [s for s in [where_lev0,where_lev1,where_lev2] if s is not None]

                qry += '''
                (
                     ''' + ' or '.join(where_nonempty) + '''  
                )         
                '''
            else:
                qry += '''
                (
                     concept_code in ('''+ ','.join(add_quote(sig['exact'])) +''')
                )         
                '''
            qry_lst.append(qry)
            
    return qry_lst